In [ ]:
from pyspark.sql.functions import col  # For column operations in Spark DataFrames
from pyspark.sql.types import TimestampType  # For casting columns to timestamp type

# ---------------------------
# Silver Layer: Cleansing and Structuring Earthquake Data
# ---------------------------

# Read the raw earthquake data (JSON) from the bronze layer into a Spark DataFrame.
# - The file path uses the start_date variable for partitioning and traceability.
# - The 'multiline' option is set to true to correctly parse JSON arrays.
df = spark.read.option("multiline", "true").json(f"Files/{start_date}_earthquake_data.json")

# Reshape earthquake data by extracting and renaming key attributes for further analysis.
# - Selects relevant fields from the nested JSON structure.
# - Renames columns for clarity and downstream analytics.
df = (
    df
    .select(
        'id',
        col('geometry.coordinates').getItem(0).alias('longitude'),
        col('geometry.coordinates').getItem(1).alias('latitude'),
        col('geometry.coordinates').getItem(2).alias('elevation'),
        col('properties.title').alias('title'),
        col('properties.place').alias('place_description'),
        col('properties.sig').alias('sig'),
        col('properties.mag').alias('mag'),
        col('properties.magType').alias('magType'),
        col('properties.time').alias('time'),
        col('properties.updated').alias('updated')
    )
)

# Convert 'time' and 'updated' columns from milliseconds to timestamp format for clearer datetime representation.
# - USGS API provides these as epoch milliseconds; Spark expects seconds for timestamp conversion.
df = (
    df
    .withColumn('time', col('time')/1000)
    .withColumn('updated', col('updated')/1000)
    .withColumn('time', col('time').cast(TimestampType()))
    .withColumn('updated', col('updated').cast(TimestampType()))
)

# Write the cleansed and structured data to the silver table.
# - Uses 'append' mode to add new data without overwriting existing records.
# - The table 'earthquake_events_silver' serves as the silver layer in the medallion architecture.
df.write.mode('append').saveAsTable('earthquake_events_silver')

# ---------------------------
# Additional Info:
# - The silver layer focuses on cleaning, normalizing, and structuring the raw data for analytics.
# - Only relevant columns are kept, and timestamps are converted for easier querying.
# - The resulting table can be used for reporting, dashboards, or further enrichment in the gold layer.
# - Ensure the Spark session is available and the input file exists before running

# Worldwide Earthquake Events API - Silver Layer Processing

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import TimestampType

In [ ]:
# df now is a Spark DataFrame containing JSON data
df = spark.read.option("multiline", "true").json(f"Files/{start_date}_earthquake_data.json")

In [ ]:
# Reshape earthquake data by extracting and renaming key attributes for further analysis.
df = \
df.\
    select(
        'id',
        col('geometry.coordinates').getItem(0).alias('longitude'),
        col('geometry.coordinates').getItem(1).alias('latitude'),
        col('geometry.coordinates').getItem(2).alias('elevation'),
        col('properties.title').alias('title'),
        col('properties.place').alias('place_description'),
        col('properties.sig').alias('sig'),
        col('properties.mag').alias('mag'),
        col('properties.magType').alias('magType'),
        col('properties.time').alias('time'),
        col('properties.updated').alias('updated')
        )

In [ ]:
# Convert 'time' and 'updated' columns from milliseconds to timestamp format for clearer datetime representation.
df = df.\
    withColumn('time', col('time')/1000).\
    withColumn('updated', col('updated')/1000).\
    withColumn('time', col('time').cast(TimestampType())).\
    withColumn('updated', col('updated').cast(TimestampType()))

In [ ]:
# appending the data to the gold table
df.write.mode('append').saveAsTable('earthquake_events_silver')